In [ ]:
!mkdir ../data
!mkdir ../data/stage1_test
!mkdir ../data/stage1_train
!unzip ../input/stage1_test.zip -d ../data/stage1_test
!unzip ../input/stage1_train.zip -d ../data/stage1_train

In [ ]:
!git clone https://github.com/MrGiovanni/UNetPlusPlus.git
# !cp -r UNetPlusPlus/segmentation_models/ segmentation_models
# !pip install -r requirements.txt
# !git submodule update --init --recursive

In [ ]:
cd UNetPlusPlus/keras

In [ ]:
!pip install -r requirements.txt
!git submodule update --init --recursive

In [ ]:
!pip install segmentation-models --quiet

In [ ]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras import backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, GlobalAveragePooling2D, Dense, Multiply, Input


# import tensorflow as tf

# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
TRAIN_PATH = '../data/stage1_train/'
TEST_PATH = '../data/stage1_test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

In [ ]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [ ]:
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [ ]:
# Check if training data looks all right
ix = random.randint(0, len(train_ids))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()

In [ ]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# Squeeze and Excitation
def se_block(input, channels, r=8):
    # Squeeze
    x = GlobalAveragePooling2D()(input)
    # Excitation
    x = Dense(channels//r, activation="relu")(x)
    x = Dense(channels, activation="sigmoid")(x)
    return Multiply()([input, x])

In [ ]:
#build se-Unet 
from keras.activations import linear

from segmentation_models import Unet,nestnet, Xnet

# prepare data
# x, y = ... # range in [0,1], the network expects input channels of 3

# prepare model
# model = Xnet(backbone_name='vgg16', encoder_weights=None, decoder_block_type='transpose',classes=1,decoder_use_batchnorm=True) # build UNet++
seUnet = Unet(backbone_name='vgg16', encoder_weights=None,freeze_encoder=False, decoder_block_type='transpose',classes=1,decoder_use_batchnorm=True) # build U-Net
# model = NestNet(backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose') # build DLA

use_se_block = True
use_batch_norm = True

x = seUnet.layers[0].input    
for i, layer in enumerate(seUnet.layers):
    if i == 0: continue
    if "decod" in layer.name:
        break
    if "conv" in layer.name:
        if use_batch_norm:
            layer.activation = linear
            x = layer(x)
#             print(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
        else:
            x = layer(x)
#             print(x)
        if use_se_block:
            x = se_block(x, layer.filters)
    else:
        x = layer(x)
# model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])
seUnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])

In [ ]:
# Fit model
epoch = 10
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
seUnet_results = seUnet.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=epoch, 
                    callbacks=[checkpointer])#earlystopper, 

In [ ]:
#build pretrained se-Unet　

from segmentation_models import Unet,nestnet, Xnet

# prepare data
# x, y = ... # range in [0,1], the network expects input channels of 3

# prepare model
# model = Xnet(backbone_name='vgg16', encoder_weights=None, decoder_block_type='transpose',classes=1,decoder_use_batchnorm=True) # build UNet++
pre_seUnet = Unet(backbone_name='vgg16', encoder_weights='imagenet',freeze_encoder=True, decoder_block_type='transpose',classes=1,decoder_use_batchnorm=True) # build U-Net
# model = NestNet(backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose') # build DLA

use_se_block = True
use_batch_norm = True

x = pre_seUnet.layers[0].input    
for i, layer in enumerate(pre_seUnet.layers):
    if i == 0: continue
    if "decod" in layer.name:
        break
    if "conv" in layer.name:
        if use_batch_norm:
            layer.activation = linear
            x = layer(x)
#             print(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
        else:
            x = layer(x)
#             print(x)
        if use_se_block:
            x = se_block(x, layer.filters)
    else:
        x = layer(x)
# model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])
pre_seUnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])

In [ ]:
# Fit model
epoch = 10
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
pre_seUnet_results = pre_seUnet.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=epoch, 
                    callbacks=[checkpointer])#earlystopper, 

In [ ]:
#build se-Unet++    

from segmentation_models import Unet,nestnet, Xnet

# prepare data
# x, y = ... # range in [0,1], the network expects input channels of 3

# prepare model
seUnetpp = Xnet(backbone_name='vgg16', encoder_weights=None,freeze_encoder=False, decoder_block_type='transpose',classes=1,decoder_use_batchnorm=True) # build UNet++
# Unet = Unet(backbone_name='vgg16', encoder_weights=None, decoder_block_type='transpose') # build U-Net
# model = NestNet(backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose') # build DLA

use_se_block = True
use_batch_norm = True

x = seUnetpp.layers[0].input    
for i, layer in enumerate(seUnetpp.layers):
    if i == 0: continue
    if "decod" in layer.name:
        break
    if "conv" in layer.name:
        if use_batch_norm:
            layer.activation = linear
            x = layer(x)
#             print(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
        else:
            x = layer(x)
#             print(x)
        if use_se_block:
            x = se_block(x, layer.filters)
    else:
        x = layer(x)

# model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])
seUnetpp.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])

In [ ]:
# Fit model
epoch = 10
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
seUnetpp_results = seUnetpp.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=epoch, 
                    callbacks=[checkpointer])#earlystopper, 

In [ ]:
#build pretrained se-Unet++    

from segmentation_models import Unet,nestnet, Xnet

# prepare data
# x, y = ... # range in [0,1], the network expects input channels of 3

# prepare model
pre_seUnetpp = Xnet(backbone_name='vgg16', encoder_weights='imagenet',freeze_encoder=True, decoder_block_type='transpose',classes=1,decoder_use_batchnorm=True) # build UNet++
# Unet = Unet(backbone_name='vgg16', encoder_weights=None, decoder_block_type='transpose') # build U-Net
# model = NestNet(backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose') # build DLA

use_se_block = True
use_batch_norm = True

x = pre_seUnetpp.layers[0].input    
for i, layer in enumerate(pre_seUnetpp.layers):
    if i == 0: continue
    if "decod" in layer.name:
        break
    if "conv" in layer.name:
        if use_batch_norm:
            layer.activation = linear
            x = layer(x)
#             print(x)
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
        else:
            x = layer(x)
#             print(x)
        if use_se_block:
            x = se_block(x, layer.filters)
    else:
        x = layer(x)

# model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])
pre_seUnetpp.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])

In [ ]:
# Fit model
epoch = 10
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
pre_seUnetpp_results = pre_seUnetpp.fit(X_train, Y_train, validation_split=0.1, batch_size=16, epochs=epoch, 
                    callbacks=[checkpointer])#earlystopper, 

In [ ]:
# seUnet vs pre_seUnet  

plt.figure()
plt.plot(range(int(epoch)),seUnet_results.history['loss'],color='blue',linestyle='-',label='seUnet_loss')
plt.plot(range(int(epoch)),seUnet_results.history['val_loss'],color='green',linestyle='--',label='seUnet_val_loss')
plt.plot(range(int(epoch)),pre_seUnet_results.history['loss'],color='red',linestyle='-',label='pretrained_seUnet_loss')
plt.plot(range(int(epoch)),pre_seUnet_results.history['val_loss'],color='orange',linestyle='--',label='pretrained_seUnet_val_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('seUnet loss')
plt.grid()

In [ ]:
plt.figure()
plt.plot(range(int(epoch)),seUnet_results.history['mean_iou'],color='blue',linestyle='-',label='seUnet_IoU')
plt.plot(range(int(epoch)),seUnet_results.history['val_mean_iou'],color='green',linestyle='--',label='seUnet_val_IoU')
plt.plot(range(int(epoch)),pre_seUnet_results.history['mean_iou'],color='red',linestyle='-',label='pre_seUnet_IoU')
plt.plot(range(int(epoch)),pre_seUnet_results.history['val_mean_iou'],color='orange',linestyle='--',label='pre_seUnet_val_IoU')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('IoU')
plt.title('seUnet IoU')
plt.grid()

In [ ]:
# seUnetpp vs pre_seUnetpp

plt.figure()
plt.plot(range(int(epoch)),seUnetpp_results.history['loss'],color='blue',linestyle='-',label='seUnet++_loss')
plt.plot(range(int(epoch)),seUnetpp_results.history['val_loss'],color='green',linestyle='--',label='seUnet++_val_loss')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['loss'],color='red',linestyle='-',label='pretrained_seUnet++_loss')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['val_loss'],color='orange',linestyle='--',label='pretrained_seUnet++_val_loss')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('seUnet++ loss')
plt.grid()

In [ ]:
plt.figure()
plt.plot(range(int(epoch)),seUnetpp_results.history['mean_iou'],color='blue',linestyle='-',label='seUnet++_IoU')
plt.plot(range(int(epoch)),seUnetpp_results.history['val_mean_iou'],color='green',linestyle='--',label='seUnet++_val_IoU')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['mean_iou'],color='red',linestyle='-',label='pre_seUnet++_IoU')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['val_mean_iou'],color='orange',linestyle='--',label='pre_seUnet++_val_IoU')
plt.legend()
plt.xlabel('epoch')
plt.ylabel('IoU')
plt.title('seUnet++ IoU')
plt.grid()

In [ ]:
#全部比較
plt.figure()
plt.plot(range(int(epoch)),seUnet_results.history['loss'],color='blue',linestyle='-',label='seU-Net_loss')
plt.plot(range(int(epoch)),seUnet_results.history['val_loss'],color='green',linestyle='--',label='seU-Net_val_loss')
plt.plot(range(int(epoch)),pre_seUnet_results.history['loss'],color='black',linestyle='-',label='Pretrained_seU-Net_loss')
plt.plot(range(int(epoch)),pre_seUnet_results.history['val_loss'],color='purple',linestyle='--',label='Pretrained_seU-Net_val_loss')
plt.plot(range(int(epoch)),seUnetpp_results.history['loss'],color='red',linestyle='-',label='seU-Net++_loss')
plt.plot(range(int(epoch)),seUnetpp_results.history['val_loss'],color='orange',linestyle='--',label='seU-Net++_val_loss')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['loss'],color='brown',linestyle='-',label='Pretrained_seU-Net++_loss')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['val_loss'],color='pink',linestyle='--',label='pretrained_seU-Net++_val_loss')

plt.legend()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('loss')
plt.grid()

In [ ]:
#全部比較
plt.figure()
plt.plot(range(int(epoch)),seUnet_results.history['mean_iou'],color='blue',linestyle='-',label='seU-Net_IoU')
plt.plot(range(int(epoch)),seUnet_results.history['val_mean_iou'],color='green',linestyle='--',label='seU-Net_val_IoU')
plt.plot(range(int(epoch)),pre_seUnet_results.history['mean_iou'],color='black',linestyle='-',label='Pretrained_seU-Net_IoU')
plt.plot(range(int(epoch)),pre_seUnet_results.history['val_mean_iou'],color='purple',linestyle='--',label='Pretrained_seU-Net_val_IoU')
plt.plot(range(int(epoch)),seUnetpp_results.history['mean_iou'],color='red',linestyle='-',label='seU-Net++_IoU')
plt.plot(range(int(epoch)),seUnetpp_results.history['val_mean_iou'],color='orange',linestyle='--',label='seU-Net++_val_IoU')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['mean_iou'],color='brown',linestyle='-',label='Pretrained_seU-Net++_IoU')
plt.plot(range(int(epoch)),pre_seUnetpp_results.history['val_mean_iou'],color='pink',linestyle='--',label='pretrained_seU-Net++_val_IoU')

plt.legend()
plt.xlabel('epoch')
plt.ylabel('IoU')
plt.title('IoU')
plt.grid()

In [ ]:
# Predict on train, val and test
model = load_model('model-dsbowl2018-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [ ]:
# Perform a sanity check on some random training samples
ix = random.randint(0, len(preds_train_t))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

In [ ]:
# Perform a sanity check on some random validation samples
ix = random.randint(0, len(preds_val_t))
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()

In [ ]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

In [ ]:
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))

In [ ]:
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('sub-dsbowl2018-1.csv', index=False)